In [1]:
import os

In [2]:
%pwd 

'/Users/rishienugala/Desktop/DSRepo/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/Users/rishienugala/Desktop/DSRepo'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_date_file: Path
    unzip_dir: Path

In [8]:
from src.datascience_project.constants import *
from src.datascience_project.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [14]:
import os
import urllib.request as request
from src.datascience_project import logger
import zipfile

In [15]:
# component-Data Ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} download! with following info: \n{headers}')
        else:
            logger.info(f"File already exists")

    def extract_online(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()

except Exception as e:
    raise e

[2026-01-19 16:05:43,631 - datasciencelogger - INFO - yaml file: /Users/rishienugala/Desktop/DSRepo/config/config.yaml loaded successfully]
[2026-01-19 16:05:43,639 - datasciencelogger - INFO - yaml file: /Users/rishienugala/Desktop/DSRepo/params.yaml loaded successfully]
[2026-01-19 16:05:43,641 - datasciencelogger - INFO - yaml file: /Users/rishienugala/Desktop/DSRepo/schema.yaml loaded successfully]


TypeError: isinstance() argument 2 cannot be a parameterized generic